In [1]:
import requests
import yaml
import pandas as pd
from bs4 import BeautifulSoup

# Character

In [2]:
NAME = 'fischl'

In [3]:
url = f'https://genshin.honeyhunterworld.com/db/char/{NAME}/'

r  = requests.get(url)

data = r.text

soup = BeautifulSoup(data)

## General Info

In [4]:
def extract_general_info(soup: BeautifulSoup) -> dict:
    weapon_type = soup.findAll(string="Weapon Type")[0].next.text
    element = soup.findAll(string='Element')[0].next.find('img').get('src').split('/')[-1].split('_')[0]
    return {'weapon_type': weapon_type, 'element':element}

In [5]:
extract_general_info(soup)

{'weapon_type': 'Bow', 'element': 'electro'}

## Extract Stat

In [6]:
def extract_stats(soup: BeautifulSoup) -> pd.DataFrame:
    table = soup.findAll(string="Stat Progression")[0].next
    
    table_list = [x.text  for i,x in enumerate(table.findAll('td')) if len(x.findAll('span')) == 0]
    column = table_list[0:7]
    
    new_table_list = table_list[8:8 + 7] + table_list[8+8:-1]
    stats = [new_table_list[i * 7: int((i + 1) * 7)] for i in range(int(len(new_table_list) / 7))]
    
    return pd.DataFrame(stats, columns=column)

In [7]:
extract_stats(soup)

,Lv,Base HP,Base ATK,Base DEF,ATK%,CRIT Rate,CRIT DMG
0,1,770,20,50,0,5,50
1,20,1979,53,128,0,5,50
2,20+,2555,68,165,0,5,50
3,40,3827,102,247,0,5,50
4,40+,4236,113,274,6,5,50
5,50,4872,130,315,6,5,50
6,50+,5418,144,350,12,5,50
7,60,6054,161,391,12,5,50
8,60+,6463,172,418,12,5,50
9,70,7099,189,459,12,5,50


## Normal Atk

In [8]:
def extract_normal_atk(soup: BeautifulSoup) -> pd.DataFrame:
    atk_spec = [x.text for x in soup.findAll('div', {'class':'skilldmgwrapper'})[0].findAll('td')]
    
    levels = atk_spec[1:16]
    
    atk_type = []
    atk_dat = []
    size =(len(levels) + 1)
    for i in range(int(len(atk_spec[16:]) / size)):
        curr = atk_spec[16:][i *size: (i+1) * size]
        atk_type.append(curr[0])
        atk_dat.append(curr[1:])
        
    return pd.DataFrame(atk_dat, columns=levels, index=atk_type)

In [9]:
extract_normal_atk(soup)

,Lv1,Lv2,Lv3,Lv4,Lv5,Lv6,Lv7,Lv8,Lv9,Lv10,Lv11,Lv12,Lv13,Lv14,Lv15
1-Hit DMG,44.12%,47.71%,51.3%,56.43%,60.02%,64.13%,69.77%,75.41%,81.05%,87.21%,93.37%,99.52%,105.68%,111.83%,117.99%
2-Hit DMG,46.78%,50.59%,54.4%,59.84%,63.65%,68%,73.98%,79.97%,85.95%,92.48%,99.01%,105.54%,112.06%,118.59%,125.12%
3-Hit DMG,58.14%,62.87%,67.6%,74.36%,79.09%,84.5%,91.94%,99.37%,106.81%,114.92%,123.03%,131.14%,139.26%,147.37%,155.48%
4-Hit DMG,57.71%,62.4%,67.1%,73.81%,78.51%,83.88%,91.26%,98.64%,106.02%,114.07%,122.12%,130.17%,138.23%,146.28%,154.33%
5-Hit DMG,72.07%,77.93%,83.8%,92.18%,98.05%,104.75%,113.97%,123.19%,132.4%,142.46%,152.52%,162.57%,172.63%,182.68%,192.74%
Aimed Shot,43.86%,47.43%,51%,56.1%,59.67%,63.75%,69.36%,74.97%,80.58%,86.7%,92.82%,98.94%,105.06%,111.18%,117.3%
Fully-Charged Aimed Shot,124%,133.3%,142.6%,155%,164.3%,173.6%,186%,198.4%,210.8%,223.2%,235.6%,248%,263.5%,279%,294.5%
Plunge DMG,56.83%,61.45%,66.08%,72.69%,77.31%,82.6%,89.87%,97.14%,104.41%,112.34%,120.27%,128.2%,136.12%,144.05%,151.98%
Low/High Plunge DMG,113.63% / 141.93%,122.88% / 153.49%,132.13% / 165.04%,145.35% / 181.54%,154.59% / 193.1%,165.17% / 206.3%,179.7% / 224.45%,194.23% / 242.61%,208.77% / 260.76%,224.62% / 280.57%,240.48% / 300.37%,256.34% / 320.18%,272.19% / 339.98%,288.05% / 359.79%,303.9% / 379.59%


## As Yaml

In [10]:
general_info = extract_general_info(soup)
normal_atk = extract_normal_atk(soup)
stats = extract_stats(soup)

In [11]:
stat_dict = stats.T.to_dict()

In [12]:
general_info

{'weapon_type': 'Bow', 'element': 'electro'}

In [13]:
single_character = {
    'name': NAME,
    'element':general_info['element'],
    'weapon_type': general_info['weapon_type'],
    'base_stats': [stat_dict[k] for k in stat_dict],
    'normal_attack': normal_atk.to_dict()
}

In [14]:
!mkdir -p ../data/example/character/

In [15]:
with open(f'../data/example/character/{NAME}.yml', 'w', encoding = "utf-8") as yaml_file:
    dump = yaml.dump(single_character, default_flow_style=False, sort_keys=False)
    yaml_file.write( dump )

## For All Chara

In [16]:
ALL_CHARA = {'albedo': 'https://genshin.honeyhunterworld.com/db/char/albedo/',
 'amber': 'https://genshin.honeyhunterworld.com/db/char/amber/',
 'ayaka': 'https://genshin.honeyhunterworld.com/db/char/ayaka/',
 'barbara': 'https://genshin.honeyhunterworld.com/db/char/barbara/',
 'beidou': 'https://genshin.honeyhunterworld.com/db/char/beidou/',
 'bennett': 'https://genshin.honeyhunterworld.com/db/char/bennett/',
 'chongyun': 'https://genshin.honeyhunterworld.com/db/char/chongyun/',
 'diluc': 'https://genshin.honeyhunterworld.com/db/char/diluc/',
 'diona': 'https://genshin.honeyhunterworld.com/db/char/diona/',
 'fischl': 'https://genshin.honeyhunterworld.com/db/char/fischl/',
 'ganyu': 'https://genshin.honeyhunterworld.com/db/char/ganyu/',
 'jean': 'https://genshin.honeyhunterworld.com/db/char/jean/',
 'kaeya': 'https://genshin.honeyhunterworld.com/db/char/kaeya/',
 'keqing': 'https://genshin.honeyhunterworld.com/db/char/keqing/',
 'klee': 'https://genshin.honeyhunterworld.com/db/char/klee/',
 'lisa': 'https://genshin.honeyhunterworld.com/db/char/lisa/',
 'mona': 'https://genshin.honeyhunterworld.com/db/char/mona/',
 'ningguang': 'https://genshin.honeyhunterworld.com/db/char/ningguang/',
 'noelle': 'https://genshin.honeyhunterworld.com/db/char/noelle/',
 'qiqi': 'https://genshin.honeyhunterworld.com/db/char/qiqi/',
 'razor': 'https://genshin.honeyhunterworld.com/db/char/razor/',
 'rosaria': 'https://genshin.honeyhunterworld.com/db/char/rosaria/',
 'sucrose': 'https://genshin.honeyhunterworld.com/db/char/sucrose/',
 'tartaglia': 'https://genshin.honeyhunterworld.com/db/char/tartaglia/',
 'traveler_anemo': 'https://genshin.honeyhunterworld.com/db/char/traveler_anemo/',
 'traveler_geo': 'https://genshin.honeyhunterworld.com/db/char/traveler_geo/',
 'venti': 'https://genshin.honeyhunterworld.com/db/char/venti/',
 'xiangling': 'https://genshin.honeyhunterworld.com/db/char/xiangling/',
 'xiao': 'https://genshin.honeyhunterworld.com/db/char/xiao/',
 'xingqiu': 'https://genshin.honeyhunterworld.com/db/char/xingqiu/',
 'xinyan': 'https://genshin.honeyhunterworld.com/db/char/xinyan/',
 'zhongli': 'https://genshin.honeyhunterworld.com/db/char/zhongli/'}

In [17]:
ALL_CHARA

{'albedo': 'https://genshin.honeyhunterworld.com/db/char/albedo/',
 'amber': 'https://genshin.honeyhunterworld.com/db/char/amber/',
 'ayaka': 'https://genshin.honeyhunterworld.com/db/char/ayaka/',
 'barbara': 'https://genshin.honeyhunterworld.com/db/char/barbara/',
 'beidou': 'https://genshin.honeyhunterworld.com/db/char/beidou/',
 'bennett': 'https://genshin.honeyhunterworld.com/db/char/bennett/',
 'chongyun': 'https://genshin.honeyhunterworld.com/db/char/chongyun/',
 'diluc': 'https://genshin.honeyhunterworld.com/db/char/diluc/',
 'diona': 'https://genshin.honeyhunterworld.com/db/char/diona/',
 'fischl': 'https://genshin.honeyhunterworld.com/db/char/fischl/',
 'ganyu': 'https://genshin.honeyhunterworld.com/db/char/ganyu/',
 'jean': 'https://genshin.honeyhunterworld.com/db/char/jean/',
 'kaeya': 'https://genshin.honeyhunterworld.com/db/char/kaeya/',
 'keqing': 'https://genshin.honeyhunterworld.com/db/char/keqing/',
 'klee': 'https://genshin.honeyhunterworld.com/db/char/klee/',
 'lisa'